<a href="https://colab.research.google.com/github/wdon021/From_Colab/blob/master/Word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
PATH = "/content/gdrive/My Drive/Fastai/"
import pandas as pd
h1b_raw = pd.read_csv(f'{PATH}H-1B_Disclosure_RAW_Data_FY18.csv', delimiter=',', engine='python')

In [0]:
job = list(h1b_raw['JOB_TITLE'].str.lower().astype('category').cat.categories)

In [0]:
from nltk import word_tokenize

In [0]:
import nltk

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
job_token = []

In [0]:
import re

In [25]:
sentences = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(job)]

TypeError: ignored

In [0]:
from gensim.models import word2vec, Phrases

In [0]:
sentence_stream = [doc.split(" ") for doc in job]

In [29]:
trigram_sentences_project = []
bigram = Phrases(sentence_stream, min_count=1, delimiter=b' ')
trigram = Phrases(bigram[sentence_stream], min_count=1, delimiter=b' ')

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [30]:
for sent in sentence_stream:
  bigrams_ = bigram[sent]
  trigrams_ = trigram[bigram[sent]]
  trigram_sentences_project.append(trigrams_)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [32]:
bigrams_

['zuora', 'subject matter', 'expert', '(payment', 'and', 'collections)']

In [12]:
model = gensim.models.Word2Vec(job, size=100, window=10, min_count=2, iter=10,workers= 4)

TypeError: ignored

In [11]:
model.train(job, total_examples=len(job), epochs=10)

(6633188, 32976060)

In [10]:
model.wv['d']

array([ 0.2909782 ,  0.13431372,  1.0106313 ,  1.3775482 ,  0.3329845 ,
       -0.7085401 ,  0.45684057,  0.60879993,  0.06423777, -0.25076243,
       -0.71982837, -0.5133624 ,  0.23305754,  1.1272373 ,  0.065159  ,
       -0.6006753 , -0.30339855, -0.27369767, -0.16146241, -0.55340666,
        1.1996415 , -0.02632387,  0.6098023 ,  0.18985446,  0.6925645 ,
        0.7084513 , -0.4839045 ,  0.3338435 , -1.7647196 , -0.95533043,
       -0.6502572 , -0.15763338, -0.7510123 , -0.42122856, -1.056317  ,
       -0.8785498 , -0.20806986, -0.96120816, -0.4606142 , -0.58940786,
        0.01498588, -1.0392706 ,  0.1990665 ,  0.33466122, -0.79947436,
       -0.2310825 ,  0.13071287, -0.4488748 , -0.542914  ,  0.52713585,
       -1.0479342 ,  0.56969494,  0.02852423, -0.25841254, -0.03070776,
       -0.42457658,  0.9144899 , -0.04501978,  0.1910019 , -0.7226273 ,
        0.29832387, -0.21808824, -0.41680345,  0.30039018,  0.36924815,
       -0.19602035,  0.5386436 , -0.7903752 ,  0.10246119,  0.79

In [0]:
word

In [0]:
model.wv.vectors

array([[ 0.11929723,  0.3678056 ,  0.05952489, ...,  0.27391693,
         0.4366963 ,  0.29420707],
       [ 0.99929094, -0.76015294,  0.00310122, ...,  0.21632332,
        -0.27968535, -0.37598258],
       [-0.41941473, -0.1189682 ,  0.39461368, ...,  0.48888555,
        -0.16593394, -0.04809809],
       ...,
       [ 0.04336734,  0.01619808,  0.03425115, ...,  0.00517803,
        -0.05098329,  0.05543505],
       [ 0.06409856,  0.02634461, -0.023928  , ...,  0.02177242,
        -0.04479816,  0.02385304],
       [-0.01888218, -0.00165699, -0.02916861, ...,  0.01988879,
        -0.00457954,  0.02924143]], dtype=float32)

In [0]:
model.wv.vectors.shape

(60, 100)

In [0]:
word_vectors